In [ ]:
# convention : all new inputs parameters for the notebbok through papermil,
# that are progressivelt added have a default value, equal to the one tha would be used in
# order obtain the same results of scripts

# two types of inputs: 
# inputs that influences the training (e.g hyper-paramteres, dataset set and splitting)
# inputs that controls state of training (reset it or load from)

batch_size= 64
d_model = 256
early_stopping_activated = False
half_period_cycle = 5
early_stop_thresh = 3*half_period_cycle
nb_epochs = 300


path_dataset = "../../data/french_english_dataset/fra.txt"
limit_length= None
use_splitting = True
path_language_info = "../../models/language_info.pth"
path_dataset_splitting = "../../dataset_splitting"
max_length_from_file = False

optimizer_option = "AdamW"

base_lr = 10**(-6)
max_lr = 0.0005
momentum=0.9

load_from_backup = True
restore_from_backup = tuple(["model_params","scheduler","optimizer","losses","metrics"])



In [ ]:
import time
import itertools


start = time.time()
min_lr_search = 10**(-7)
max_lr_search = 0.1
size_experiments_step = 2000
cycle_schedule = 5
method = "exponential"

def lr_range_test(self,min_lr_search,max_lr_search,size_experiments_step,cycle_schedule):
    total_iters = size_experiments_step//cycle_schedule

    if method=="exponential":
        gamma = np.exp(np.log(max_lr_search/min_lr_search)/total_iters)
        optimizer = torch.optim.NAdam(model.parameters(), lr=min_lr_search)
        scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer,gamma=gamma)
    elif method == "linear":
        optimizer = torch.optim.NAdam(model.parameters(), lr=max_lr_search)
        scheduler = torch.optim.lr_scheduler.LinearLR(optimizer,start_factor=0.0001,end_factor=1.0,total_iters=total_iters)
    else:
        raise ValueError

    self.optimizer = optimizer


    cyclic_shuffled_train_data_loader = (el  for _ in itertools.count(start=0,step=1) for el in train_data_loader)
    cyclic_shuffled_train_data_loader = itertools.islice(cyclic_shuffled_train_data_loader,size_experiments_step)

    indexed_cyclic_shuffled_train_data_loader = enumerate(cyclic_shuffled_train_data_loader)
    #losses_lr_finder = []
    lrs = []

    for idx,batch in indexed_cyclic_shuffled_train_data_loader:
        print(idx)
        loss,nb_words = model_trainer.train_on_batch(batch)
        loss = float(loss)
        if idx%cycle_schedule==0:
            print(idx,scheduler.get_last_lr())
            scheduler.step()
        print(loss/nb_words,scheduler.get_last_lr())
        losses_lr.append(loss/nb_words)
        lrs.append(scheduler.get_last_lr())
    return losses_lr,lrs

lrs = lr_range_test(model_trainer,min_lr_search,max_lr_search,size_experiments_step,cycle_schedule)

In [ ]:
tmp = np.mean(np.array(losses_lr_finder_exponential).reshape(-1,5),axis=1)
plt.plot(tmp)